In [1]:
import pandas as pd
df = pd.read_csv('Merge3.csv')
df.head()
from io import StringIO
col = ['Category', 'Write']
df = df[col]
df = df[pd.notnull(df['Write'])]
df.columns = ['Category', 'Write']
df['category_id'] = df['Category'].factorize()[0]
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)
df.head()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 2), stop_words=None)
features = tfidf.fit_transform(df.Write).toarray()
labels = df.category_id
features.shape
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for Category, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Category))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['Write'], df['Category'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
print(clf.predict(count_vect.transform(["This company refuses to provide me verification and validation of debt per my right under the FDCPA. I do not believe this debt is mine."])))
df[df['Write'] == "This company refuses to provide me verification and validation of debt per my right under the FDCPA. I do not believe this debt is mine."]
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
cv_df.groupby('model_name').accuracy.mean()

# 'Accident':
  . Most correlated unigrams:
. থল
. উপজ
  . Most correlated bigrams:
. ঘটন ঘট
. ঘটন থল
# 'Art':
  . Most correlated unigrams:
. সমক
. ছদ
  . Most correlated bigrams:
. অভ নয়
. রচ ছদ
# 'Crime':
  . Most correlated unigrams:
. ইউএনও
. নজনক
  . Most correlated bigrams:
. কর গতক
. গতক শন
# 'Economics':
  . Most correlated unigrams:
. এসই
. এসইত
  . Most correlated bigrams:
. এসই চক
. পয় হয়
# 'Education':
  . Most correlated unigrams:
. জনশ
. ১ম
  . Most correlated bigrams:
. চন রশ
. রশ তর
# 'Entertainment':
  . Most correlated unigrams:
. যচ
. চলচ
  . Most correlated bigrams:
. অভ নয়
. অভ নয
# 'Environment':
  . Most correlated unigrams:
. দরবন
. খনন
  . Most correlated bigrams:
. অধ দপ
. নদ রব
# 'International':
  . Most correlated unigrams:
. পরম
. আইএস
  . Most correlated bigrams:
. ২০১৮ বক
. ইসল আইএস
# 'Opinion':
  . Most correlated unigrams:
. অবম
. জনকণ
  . Most correlated bigrams:
. মত মত
. অবম নন
# 'Politics':
  . Most correlated unigrams:
. আওয়
. এনপ
  . Most correl

model_name
LinearSVC                 0.683333
LogisticRegression        0.614583
MultinomialNB             0.512500
RandomForestClassifier    0.462500
Name: accuracy, dtype: float64